In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:1

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [4]:
total_votes_df = df.filter(df['total_votes'] >= 20)
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...|2015-08-31 00:00:00|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...| 

In [5]:
helpful_votes_df = df.filter(df['helpful_votes']/df['total_votes'] >= 0.5)
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2963837|R23H79DHOZTYAU|B0081EH12M|     770100932|New Trigger Grips...|     Video Games|          1|            1|          1|   N|                Y|Now i have to buy...|It did not fit th...|2015-08-31 00:00:00|
|         US|    1581810|R3AACSKX1EAHUY|B001REZLY8|     816407837|Fallout 3: Game o...| 

In [6]:
vine_review_df = helpful_votes_df.filter(df['vine'] == 'Y')
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12113794| RWKQQPG100CQ4|B00YY3ODTS|     975991516|SteelSeries Siber...|     Video Games|          4|            2|          4|   Y|                N|iwouldcomeupwitha...|How ironic is it ...|2015-08-28 00:00:00|
|         US|   39916916|R1R0J8S1VGALUY|B00UOYQ5DS|     166248134|Razer Serval - Mo...| 

In [7]:
no_vine_review_df = helpful_votes_df.filter(df['vine'] == 'N')
no_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2963837|R23H79DHOZTYAU|B0081EH12M|     770100932|New Trigger Grips...|     Video Games|          1|            1|          1|   N|                Y|Now i have to buy...|It did not fit th...|2015-08-31 00:00:00|
|         US|    1581810|R3AACSKX1EAHUY|B001REZLY8|     816407837|Fallout 3: Game o...| 

In [8]:
#Number of 5 star reviews
total_5_star = helpful_votes_df.filter(df['star_rating'] == 5).count()
total_5_star

291835

In [9]:
#Total Number of reviews greater or equal to 20
total_helpful_reviews = helpful_votes_df.count()
total_helpful_reviews

574168

In [10]:
#Total number of paid vine reviews
paid = vine_review_df.count()
paid

1757

In [11]:
#Total number of unpaid vine reviews
unpaid = no_vine_review_df.count()
unpaid

572411

In [13]:
#Number of paid 5 star vine reviews
vine_review_5_star = vine_review_df.filter(df['star_rating'] == 5).count()
vine_review_5_star

700

In [20]:
# Percent of 5 Star vine reviews
pct_of_5_star_vine = vine_review_5_star/paid * 100
print(f'{round(pct_of_5_star_vine,2)}%')

39.84%


In [21]:
# Percent of 5 Star non-vine reviews
pct_of_5_star_non_vine = no_vine_review_5_star/unpaid * 100
print(f'{round(pct_of_5_star_non_vine,2)}%')

50.86%


In [ ]:
#Percentage of 5 Star paid reviews
pct_of_paid_5_star = round(vine_review_5_star/total_helpful_reviews * 100,2)
print(f'{pct_of_paid_5_star}%') 

0.12%


In [18]:
#Number of unpaid 5 star vine reviews
no_vine_review_5_star = no_vine_review_df.filter(df['star_rating'] == 5).count()
no_vine_review_5_star

291135

In [ ]:
#Percentage of 5 Star unpaid reviews
pct_of_paid_5_star = round(no_vine_review_5_star/total_helpful_reviews * 100,2)
print(f'{pct_of_paid_5_star}%') 

50.71%


In [ ]:
# Percentage of paid 5 star vine reviews vs Total 5 Star reviews
pct_of_5_star_vine_paid = round(vine_review_5_star/total_5_star * 100, 2)
print(f'{pct_of_5_star_vine_paid}%') 


0.24%


In [ ]:
# Percentage of unpaid 5 star vine reviews Total 5 Star reviews
pct_of_5_star_vine_unpaid = round(no_vine_review_5_star/total_5_star * 100, 2)
print(f'{pct_of_5_star_vine_unpaid}%') 

99.76%
